# Converter

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
from converter import convert_func
DATA_FOLDER = "./data/"
CAM_NAME = "camera3"

# DATA_FOLDER = "./data2/"
# CAM_NAME = "camera2"

# DATA_FOLDER = "./data0/"
# CAM_NAME = "camera0"
%matplotlib inline

In [ ]:
# Load optim function
json_fname = f"convert_params_{CAM_NAME}.json"
with open(json_fname) as f:
    params = json.load(f)
    
print(params)

In [ ]:
# Load image without gamma noise
wo_noise = cv2.imread(f"{DATA_FOLDER}/cam3_wo_noise.png", 0)
# wo_noise = cv2.imread(f"{DATA_FOLDER}/image_wo_noise.png", 0)

plt.imshow(wo_noise, vmin=0, vmax=255)

In [ ]:
# Generate gamma noise image
noisy_imgs = []
for i in range(5):
    geant4_deposit = np.load(f"{DATA_FOLDER}/geant4_gamma_noise_{i}.npy")
    noisy_img = convert_func(geant4_deposit, wo_noise, **params)
    noisy_imgs.append(noisy_img)

In [ ]:
from scipy.stats import binom
# Salt-and-pepper noise
img_size = wo_noise.shape
p = 0.01
salt_pepper = binom.rvs(n=1, p=p, loc=0, size=img_size)
plt.imshow(salt_pepper)

In [ ]:
# Salt-and-pepper
json_fname = f"convert_params_{CAM_NAME}_salt_pepper.json"
with open(json_fname) as f:
    sp_params = json.load(f)
    
def add_salt_and_pepper(wo_noise, p=0.1):
    s_p_noisy = wo_noise.copy()
    salt_pepper = binom.rvs(n=1, p=p, loc=0, size=img_size)
    s_p_noisy[salt_pepper==1] = 255
    return s_p_noisy

In [ ]:
# Visualize gamma images
from itertools import cycle
i = 0
while True:
    noisy = noisy_imgs[i%len(noisy_imgs)]
    cv2.imshow("img wo noise", wo_noise)
    cv2.imshow("noisy img", noisy)
    
    s_p_noisy = add_salt_and_pepper(wo_noise, p=sp_params['prob'])
    cv2.imshow("Salt-and-pepper noise", s_p_noisy)

    key = cv2.waitKey(100)
    if key == 27:
        break
    i += 1
cv2.destroyAllWindows()

In [ ]:
len(noisy_imgs)

In [ ]:
# s_p_imgs = [add_salt_and_pepper(wo_noise, p=sp_params["prob"]) for _ in range(5)]
# PAPER_FOLDER = f"./{CAM_NAME}_noise_paper/"
# for i, img in enumerate(s_p_imgs):
#     cv2.imwrite(f"{PAPER_FOLDER}/sp_noise_img_{i}.png", img)
# print(sp_params)

# Metrics

In [ ]:
# PAPER_FOLDER = f"./{CAM_NAME}_noise_paper/"
# print(PAPER_FOLDER)
# proposed_imgs = [cv2.imread(f"{PAPER_FOLDER}/generated_noise_img_{i}.png", 0) for i in range(5)]
# s_p_imgs = [cv2.imread(f"{PAPER_FOLDER}/sp_noise_img_{i}.png", 0) for i in range(5)]
# gt_noisy_imgs = [cv2.imread(f"{PAPER_FOLDER}/gt_noise_img_{i}.png", 0) for i in range(5)]

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# from skimage import img_as_float
# from skimage.metrics import structural_similarity as ssim

# img = img_as_float(gt_noisy_imgs[0])
# img_proposed = img_as_float(proposed_imgs[0])
# img_sp = img_as_float(s_p_imgs[0])

# fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 4),
#                          sharex=True, sharey=True)
# ax = axes.ravel()

# ssim_proposed = ssim(img, img_proposed)
# ssim_sp = ssim(img, img_sp)

# label = 'SSIM: {:.2f}'

# ax[0].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
# ax[0].set_title('Groudtruth image')

# ax[1].imshow(img_proposed, cmap=plt.cm.gray, vmin=0, vmax=1)
# ax[1].set_xlabel(label.format(ssim_proposed))
# ax[1].set_title('Proposed noisy image')

# ax[2].imshow(img_sp, cmap=plt.cm.gray, vmin=0, vmax=1)
# ax[2].set_xlabel(label.format(ssim_sp))
# ax[2].set_title('Salt and pepper noisy image')

# plt.tight_layout()
# plt.savefig(f"{PAPER_FOLDER}/plot_sim.pdf")
# plt.show()


In [ ]:
# msg = '[{}] SSIM: {:.2f}'
# # Proposed method score
# a = gt_noisy_imgs
# b = proposed_imgs
# proposed_scores = []
# for gt, img in ((x,y) for x in a for y in b):
#     gt = img_as_float(gt)
#     img = img_as_float(img)
#     score = ssim(gt, img)
#     proposed_scores.append(score)
# print(msg.format("proposed", np.average(proposed_scores)))

# # Salt-and-pepper method score
# a = gt_noisy_imgs
# b = s_p_imgs
# sp_scores = []
# for gt, img in ((x,y) for x in a for y in b):
#     gt = img_as_float(gt)
#     img = img_as_float(img)
#     score = ssim(gt, img)
#     sp_scores.append(score)
    
# print(msg.format("salt-and-pepper", np.average(sp_scores)))

In [ ]:
# with open(f"{PAPER_FOLDER}/ssim_score.txt", 'w') as f:
#     msgs = [msg.format("proposed", np.average(proposed_scores)),
#             msg.format("salt-and-pepper", np.average(sp_scores))]
#     f.writelines("\n".join(msgs))